In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os

from utils.double_ml import *
from utils.analysis import *

In [2]:
RANDOM_SEED = 42

### Import data

In [3]:
wf2020 = make_wf2020()

fixed = ['treat']
city_fixed = []
time_fixed = []
for col in wf2020.columns:
    if 'cities' in col:
        city_fixed.append(col)
    if 'days' in col:
        time_fixed.append(col)
fixed = fixed + city_fixed + time_fixed
        
weather = ['prec', 'snow', 'temp', 'temp2']
city_economic = ['pop_city', 'sec_city', 'gdp_city' , 'pgdp_city', 'firm_city']
city_environmental = ['gonglu', 'emit_ww', 'emit_so1', 'emi_dust1']
out = ["aqi", "l_aqi", "pm", "l_pm"]

In [4]:
day, count, num_cities = get_day_count(wf2020)
treat_day = day[count == max(count)][0]

### Single time period

In [5]:
tau_hat, std_hat = single_period_estimate(wf2020, treat_day=8426, outcome_var='aqi', 
                                          confounder_list=weather + city_fixed + time_fixed,
                                          Q_model_class=LinearRegression, Q_model_params={},
                                          g_model_class=LogisticRegression, g_model_params={'max_iter':1000})

print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

/Users/jtf/anaconda3/envs/mamba-base-env/envs/nn_class/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


The estimate is -35.04651188879948 pm 11.730398758415467


### Multiple time periods

In [6]:
wf2020 = make_wf2020()
tau_hat, std_hat = multi_period_estimate(wf2020, outcome_var='aqi', 
                                         confounder_list=weather + city_fixed + time_fixed,
                                         Q_model_class=RandomForestRegressor, Q_model_params={
                                             'random_state': RANDOM_SEED,
                                             'n_estimators': 100,
                                             'max_depth': 10
                                         },
                                         g_model_class=RandomForestClassifier, g_model_params={
                                             'random_state': RANDOM_SEED,
                                             'n_estimators': 100,
                                             'max_depth': 3
                                         }
                                         )
print('%0.3f pm %0.3f' % (tau_hat, std_hat))

-9.556 pm 2.947


In [42]:
dummy = wf2020['pastweek_mean'] = wf2020.groupby('city_code')['aqi'].transform(
        lambda x: pd.Series.rolling(window=7, min_periods=7).mean()
    )

In [46]:
wf2020['pastweek_mean'] = wf2020.groupby(['daynum', 'city_code']).rolling(window=7, min_periods=7)['aqi'].mean().reset_index()

ValueError: Columns must be same length as key

In [28]:
wf2020.groupby('daynum')['daynum'].apply(display)

365       8401
804       8401
1243      8401
1682      8401
2121      8401
          ... 
142162    8401
142601    8401
143040    8401
143479    8401
144357    8401
Name: 8401, Length: 324, dtype: int64

366       8402
805       8402
1244      8402
1683      8402
2122      8402
          ... 
142163    8402
142602    8402
143041    8402
143480    8402
144358    8402
Name: 8402, Length: 324, dtype: int64

367       8403
806       8403
1245      8403
1684      8403
2123      8403
          ... 
142164    8403
142603    8403
143042    8403
143481    8403
144359    8403
Name: 8403, Length: 324, dtype: int64

368       8404
807       8404
1246      8404
1685      8404
2124      8404
          ... 
142165    8404
142604    8404
143043    8404
143482    8404
144360    8404
Name: 8404, Length: 324, dtype: int64

369       8405
808       8405
1247      8405
1686      8405
2125      8405
          ... 
142166    8405
142605    8405
143044    8405
143483    8405
144361    8405
Name: 8405, Length: 324, dtype: int64

...

422       8458
861       8458
1300      8458
1739      8458
2178      8458
          ... 
142219    8458
142658    8458
143097    8458
143536    8458
144414    8458
Name: 8458, Length: 324, dtype: int64

423       8459
862       8459
1301      8459
1740      8459
2179      8459
          ... 
142220    8459
142659    8459
143098    8459
143537    8459
144415    8459
Name: 8459, Length: 324, dtype: int64

424       8460
863       8460
1302      8460
1741      8460
2180      8460
          ... 
142221    8460
142660    8460
143099    8460
143538    8460
144416    8460
Name: 8460, Length: 324, dtype: int64

425       8461
864       8461
1303      8461
1742      8461
2181      8461
          ... 
142222    8461
142661    8461
143100    8461
143539    8461
144417    8461
Name: 8461, Length: 324, dtype: int64

daynum
8401    None
8402    None
8403    None
8404    None
8405    None
        ... 
8457    None
8458    None
8459    None
8460    None
8461    None
Name: daynum, Length: 61, dtype: object